# 2nd Level Model Structure: Reactive Stage

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from physics.kinetics_bounded import kinetic_block_rule
from physics.energy_bounded import energy_block_rule
from physics.VLE_bounded import VLE_block_rule

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

# Global Variables

In [4]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,300+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.cat = pe.Var(within=pe.NonNegativeReals,initialize=3000) # kg
model.Q_main = pe.Var(within=pe.Reals) # MW
model.r_total_comp = pe.Var(m.COMP_TOTAL,within=pe.Reals) # kmol/s

# MPCC construction
model.beta = pe.Var(within=pe.NonNegativeReals,initialize=1)
model.s_L= pe.Var(within=pe.NonNegativeReals,initialize=0,bounds=(0,1))
model.s_V= pe.Var(within=pe.NonNegativeReals,initialize=0)

# MPCC Parameters

In [5]:
model.epi = pe.Param(initialize=1e-4,mutable=True)

# Construct Individual Blocks

In [6]:
model.kinetics_block = pe.Block(rule=kinetic_block_rule)

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| kinetics_block.k_FT
| kinetics_block.r_FT_total
| kinetics_block.g0_FT
| kinetics_block.alpha
| kinetics_block.r_FT_cnum
| kinetics_block.r_FT_comp
| kinetics_block.k_WGS
| kinetics_block.Ke_WGS
| kinetics_block.r_WGS
| kinetics_block.r_WGS_comp
--------------------------------------------------



In [7]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [8]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}+\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [9]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [10]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.beta*model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

## MPCC: Beta

In [11]:
def beta_rule(model):
    return model.beta == 1-model.s_L + model.s_V
model.beta_con = pe.Constraint(rule=beta_rule)

def s_L_complementarity_rule(model):
    return (sum(model.L[s] for s in model.outlet) + model.s_L) == ((sum(model.L[s] for s in model.outlet) - model.s_L)**2+model.epi)**0.5
    # return sum(model.L[s] for s in model.outlet) * model.s_L <= model.epi
model.s_L_complementarity_con = pe.Constraint(rule=s_L_complementarity_rule)

def s_V_complementarity_rule(model):
    return (sum(model.V[s] for s in model.outlet) + model.s_V) == ((sum(model.V[s] for s in model.outlet) - model.s_V)**2+model.epi)**0.5
    # return sum(model.V[s] for s in model.outlet) * model.s_V <= model.epi
model.s_V_complementarity_con = pe.Constraint(rule=s_V_complementarity_rule)

## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [12]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet) + sum(model.r_total_comp[i] for i in m.COMP_TOTAL)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [13]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [14]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


## Fixing In/Product Redundent Flow

In [15]:
model.x_.fix(0)
model.y_.fix(0)
model.L['in'].fix(0)
model.L['P'].fix(0)
model.V['in'].fix(0)
model.V['P'].fix(0)
model.H_L_.fix(0)
model.H_V_.fix(0)

## Fixing Model Parameters

In [16]:
model.cat.fix(300)
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(1)
model.z['CO'].fix(1/(1+0.8)-0/2)
model.z['H2'].fix(0.8/(1+0.8)-0/2)
model.z['C30H62'].fix(0)
model.VLE_block.n_ave.fix(20)

In [17]:
check_DOF(pe,model)

Active Equality Constraints:	 1184
Active Inequality Constraints:	 0
Active Variables:		 1357
Fixed Variables:		 172
DOF:				 1


## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [18]:
model.T.setub(520)

In [19]:
# model.obj = pe.Objective(expr = model.L['out'],sense=pe.maximize)
# model.obj = pe.Objective(expr = model.cat,sense=pe.minimize)
model.obj = pe.Objective(expr = model.T ,sense=pe.maximize)
# model.obj = pe.Objective(expr = model.T - sum(model.L[s] for s in model.outlet)*model.s_L ,sense=pe.maximize)

In [20]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
# opt.options['linear_solver'] = 'ma57'
# opt.options['ma57_automatic_scaling'] = 'no'
# opt.options['nlp_scaling_method'] = None
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
max_iter=7000


List of user-set options:

                                    Name   Value                used
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4016
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:

  84r-5.1999978e+02 1.86e+01 1.21e+03   0.4 2.31e+03    -  8.35e-03 3.69e-04f  1
  85 -5.1999977e+02 1.86e+01 3.03e+03  -1.0 1.58e+02    -  2.78e-02 4.53e-05h  1
  86r-5.1999977e+02 1.86e+01 1.00e+03   0.3 0.00e+00    -  0.00e+00 4.60e-07R  4
  87r-5.1994967e+02 1.51e+01 1.25e+03   0.3 2.17e+04    -  9.67e-03 3.50e-04f  1
  88r-5.1995171e+02 5.82e+00 1.38e+03   0.3 2.77e+02    -  1.17e-02 5.52e-03f  1
  89r-5.1995125e+02 2.56e+00 1.18e+03   0.3 2.29e+02    -  6.90e-04 1.02e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90r-5.1995073e+02 2.01e+00 2.31e+03   0.3 2.10e+02    -  7.12e-02 3.89e-03f  1
  91r-5.1994116e+02 2.17e+00 2.48e+03   0.3 2.67e+02    -  6.72e-02 4.25e-02f  1
  92r-5.1993499e+02 2.27e+00 3.12e+03   0.3 3.20e+02    -  1.08e-01 1.95e-02f  1
  93r-5.1989971e+02 3.07e+00 4.72e+03   0.3 2.56e+02    -  1.96e-01 6.93e-02f  1
  94r-5.1933976e+02 3.08e+00 5.82e+03   0.3 1.66e+03    -  1.64e-01 5.31e-02f  1
  95r-5.1890826e+02 3.30e+00

 178 -5.0551035e+02 1.04e+00 3.06e+09  -1.0 3.78e+03    -  1.00e+00 6.65e-03h  7
 179 -5.0533949e+02 1.04e+00 3.86e+09  -1.0 3.74e+03    -  2.14e-01 6.68e-03h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180 -5.0516961e+02 1.03e+00 7.60e+09  -1.0 3.70e+03    -  7.66e-01 6.71e-03h  7
 181 -5.0500073e+02 1.02e+00 1.02e+10  -1.0 3.65e+03    -  2.76e-01 6.74e-03h  7
 182 -4.9425491e+02 4.59e+01 8.14e+11  -1.0 3.61e+03    -  1.83e-01 4.33e-01w  1
 183 -4.9174073e+02 3.52e+01 5.10e+11  -1.0 1.17e+03    -  5.87e-01 2.84e-01w  1
 184 -4.9166594e+02 3.46e+01 4.97e+11  -1.0 4.99e+02    -  1.65e-02 1.90e-02w  1
 185 -5.0466492e+02 1.01e+00 1.22e+10  -1.0 2.71e+02    -  1.83e-01 1.35e-02h  5
 186 -5.0433307e+02 1.00e+00 2.66e+10  -1.0 3.53e+03    -  1.00e+00 1.37e-02h  6
 187 -5.0416913e+02 9.96e-01 2.75e+10  -1.0 3.44e+03    -  4.03e-02 6.92e-03h  7
 188 -5.0410352e+02 9.93e-01 4.78e+10  -1.0 3.40e+03    -  6.21e-01 2.80e-03h  8
 189 -5.0405558e+02 9.91e-01

In [21]:
model.T.value

520.0

In [22]:
model.beta.value

0.361896924673867

In [23]:
model.s_L.value

0.6381394592256806

In [24]:
model.L['out'].value

3.917638948444825e-05

# epi = 1e-5

In [25]:
model.epi = 1e-5

In [26]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
max_iter=7000


List of user-set options:

                                    Name   Value                used
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4016
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:

In [27]:
model.T.value

520.0

In [28]:
model.beta.value

0.49202811699895094

In [29]:
model.s_L.value

0.5079755211833071

In [30]:
model.L['out'].value

4.921496992998416e-06

# epi = 1e-6

In [31]:
model.epi = 1e-6

In [32]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
max_iter=7000


List of user-set options:

                                    Name   Value                used
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4016
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:

In [33]:
model.T.value

520.0

In [34]:
model.beta.value

0.6160067929772846

In [35]:
model.s_L.value

0.3839935708383389

In [36]:
model.L['out'].value

6.510525669520658e-07

# epi = 1e-7

In [37]:
model.epi = 1e-7

In [38]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
max_iter=7000


List of user-set options:

                                    Name   Value                used
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4016
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:

In [39]:
model.T.value

520.0

In [40]:
model.beta.value

0.7123547353055278

In [41]:
model.s_L.value

0.2876453010760001

In [42]:
model.L['out'].value

8.691259651635415e-08

# epi = 1e-8

In [43]:
model.epi = 1e-8

In [44]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
max_iter=7000


List of user-set options:

                                    Name   Value                used
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4016
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:

In [45]:
model.T.value

520.0

In [46]:
model.beta.value

0.7529305088421685

In [47]:
model.s_L.value

0.24706949479598384

In [48]:
model.L['out'].value

1.011861057281243e-08

# epi = 1e-12

In [49]:
model.epi = 1e-12

In [50]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
max_iter=7000


List of user-set options:

                                    Name   Value                used
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4016
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:

In [51]:
model.T.value

520.0

In [52]:
model.beta.value

0.7607467231624826

In [53]:
model.s_L.value

0.2392532768378811

In [54]:
model.L['out'].value

1.0449254377640906e-12

In [55]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}'.format(model.x[i].value,model.y[i].value))

Component		Liquid: 0.0000			Vapor: 0.6872
------------------------------------------------------------------------------------------------------------
H2         		 1.8828%			35.2205%
CO         		 2.2939%			34.5014%
CO2        		 3.1004%			22.0790%
H2O        		 0.5893%			 2.1884%
C2H4       		 0.1267%			 0.7284%
C3H6       		 0.2326%			 0.6741%
C4H8       		 0.1694%			 0.4819%
C5H10      		 0.2045%			 0.3003%
C6H12      		 0.2709%			 0.2259%
C7H14      		 0.3378%			 0.1699%
C8H16      		 0.4039%			 0.1279%
C9H18      		 0.4685%			 0.0962%
C10H20     		 0.5309%			 0.0724%
C11H22     		 0.3765%			 0.0347%
C12H24     		 0.4130%			 0.0261%
C13H26     		 0.4476%			 0.0196%
C14H28     		 0.4804%			 0.0148%
C15H30     		 0.5113%			 0.0111%
C16H32     		 0.5403%			 0.0084%
C17H34     		 0.5674%			 0.0063%
C18H36     		 0.5928%			 0.0047%
C19H38     		 0.6164%			 0.0036%
C20H40     		 0.6384%			 0.0027%
C1H4       		 0.1665%			 1.4887%
C2H6       		 0.0751%			 0.3916%
C3H8       		 0.0574%			

In [ ]:
# epi = 1e-8

model.epi = 1e-8

results = opt.solve(model,tee=True)
update_dual(pe,model)

model.T.value

model.s_L.value

model.L['out'].value

print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}'.format(model.x[i].value,model.y[i].value))

# Iterative Solve for Data Analysis

In [ ]:
update_dual(pe,model)

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
rf_data = {}
rf_data['r'] = {}; rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = []; rf_data['r_WGS'] = []; rf_data['r_FT'] = []
rf_data['f_V_CO'] = []; rf_data['f_V_H2'] = []; rf_data['s_L'] = []; rf_data['s_V'] = []; rf_data['beta'] = [];

for i in model.r_total_comp:
    rf_data['r'][i] = []
    
Trange = np.arange(300+273.15,199+273.15,-2)

for Tub in Trange:
    model.T.setub(Tub)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    # print('-'*72)
    print('Solved, Solution T = {0:.2f} K|\tV = {1:.5f}|\tL = {2:.5f}|\ts_L = {3:.2f}|\tbeta={4:.2f}'.format(model.T.value,model.V['out'].value,model.L['out'].value,model.s_L.value,model.beta.value))
    # print('-'*72)
    rf_data['T'].append(model.T.value)
    rf_data['Q'].append(model.Q_main.value)
    rf_data['V'].append(model.V['out'].value)
    rf_data['L'].append(model.L['out'].value)    

    for i in model.r_total_comp:
        rf_data['r'][i].append(model.r_total_comp[i].value)
    
    
    rf_data['y_H2O'].append(model.y['H2O'].value)
    rf_data['y_CO'].append(model.y['CO'].value)
    rf_data['y_H2'].append(model.y['H2'].value)
    rf_data['y_CO2'].append(model.y['CO2'].value)
    rf_data['r_WGS'].append(model.kinetics_block.r_WGS.value)
    rf_data['r_FT'].append(model.kinetics_block.r_FT_total.value)
    
    rf_data['f_V_CO'].append(model.f_V['CO'].value)
    rf_data['f_V_H2'].append(model.f_V['H2'].value)
    rf_data['s_L'].append(model.s_L.value)
    rf_data['s_V'].append(model.s_V.value)
    rf_data['beta'].append(model.beta.value)

# Optimize across disappearing phases

In [ ]:
model.T.fixed = True
model.cat.fixed = False
model.cat.setlb(1)

In [ ]:
check_DOF(pe,model)

In [ ]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = model.cat, sense=pe.minimize)

In [ ]:
results = opt.solve(model,tee=True)
update_dual(pe,model)